### Install Postgres on Raspberry Pi

Follow the instructions in this article: [Setting up a PostgreSQL Database on a Raspberry Pi](https://pimylifeup.com/raspberry-pi-postgresql/)


In [1]:
%%bash
sudo apt update -y
sudo apt install -y postgresql

Hit:1 https://download.docker.com/linux/raspbian buster InRelease
Hit:2 http://archive.raspberrypi.org/debian buster InRelease
Hit:3 http://raspbian.raspberrypi.org/raspbian buster InRelease
Reading package lists...
Building dependency tree...
Reading state information...
98 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
postgresql is already the newest version (11+200+deb10u4).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [2]:
%%html
<img src="secure-postgres-8x.gif" width="1200px" />

### Install nginx

In [1]:
%%bash
sudo apt install -y nginx
sudo service nginx status

● nginx.service - A high performance web server and a reverse proxy server
   Loaded: loaded (/lib/systemd/system/nginx.service; enabled; vendor preset: enabled)
   Active: active (running) since Sun 2022-05-29 00:19:26 PDT; 1 day 14h ago
     Docs: man:nginx(8)
 Main PID: 562 (nginx)
    Tasks: 5 (limit: 4915)
   CGroup: /system.slice/nginx.service
           ├─ 562 nginx: master process /usr/sbin/nginx -g daemon on; master_process on;
           ├─1206 nginx: worker process
           ├─1207 nginx: worker process
           ├─1208 nginx: worker process
           └─1209 nginx: worker process

May 29 00:19:25 raspberrypi systemd[1]: Starting A high performance web server and a reverse proxy server...
May 29 00:19:26 raspberrypi systemd[1]: Started A high performance web server and a reverse proxy server.


### Enable SSL

Open ports 80 and 443 on your router, and forward them to Raspberry Pi

<img src="eero-raspberry-pi.png" width="400px">
<img src="eero-web-80.png" width="400px">
<img src="eero-web-443.png" width="400px">

#### Map a domain name to your external IP address using Route 53

In [3]:
# Find out your external IP address
!curl ipconfig.io/ip

71.202.157.221


<img src="route53-create-record.png" width="600px">

#### Confirm that you are able to reach your web server using the domain name

If XFinity is your Internet service provider, this may only work from OUTSIDE the local network. Switch to your mobile Internet connection before proceeding

eg. http://pi400.cloudmatica.com


#### Upgrade your connection to https by running certbot

In [4]:
!sudo apt install -y certbot python-certbot-nginx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
certbot is already the newest version (0.31.0-1+deb10u1).
python-certbot-nginx is already the newest version (0.31.0-1).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [ ]:
!sudo certbot --nginx --domains pi400.cloudmatica.com

<img src="certbot.gif" width="800px">